<a href="https://colab.research.google.com/github/rubyvanrooyen/katcomm/blob/master/Users/ruby/oh_masers/G330/G330_CASA_calibration_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration strategy

* Set flux on primary calibrator
* Delay calibration
* Bandpass calibration stablised with phase calibration
* Complex gain calibration using flux calibrator
* Complex gain calibration of other primary calibrators
* Complex gain calibration of secondary calibrators


Start data inspection on primary calibrator, make note of suitably flat channel range to use for preliminary gain calibration prior to bandpass calibration. Confirm reference antenna looks ok

In [ ]:
def check_selection(msfile, calibrator, c_start=0, c_stop=0, r_ant=''):
    print("Quick data inspection to confirm ref ant and ref chans before calibration")
    # verify or make note of suitably flat channel range to use for preliminary gain calibration prior to bandpass calibration
    # min rfi
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='amp', correlation='XX,YY', field='{calibrator}', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[{c_start}, {c_stop}, 0, 0])")
    # flat phase
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{calibrator}', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[{c_start}, {c_stop}, -180, 180])")

    # confirm reference antenna looks ok
    if r_ant:
        avg_bl='False'
    else:
        avg_bl='True'
    # verify now bad data like slew in scan
    print(f"plotms(vis='{msfile}', xaxis='time', yaxis='amp', correlation='XX,YY', field='{calibrator}', antenna='{r_ant}', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline={avg_bl}, coloraxis='corr')")
    # flat phase over entire time range
    print(f"plotms(vis='{msfile}', xaxis='time', yaxis='phase', correlation='XX,YY', field='{calibrator}', antenna='{r_ant}', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline={avg_bl}, coloraxis='corr')")
    print("Switch views interactively as needed, check other sources for obvious issues that need to be flagged.")

https://github.com/rubyvanrooyen/katcomm/blob/master/Users/ruby/oh_masers/G330/G330_CASA_data_inspection_commands.ipynb

Helper functions

In [ ]:
def _get_prefix_(msfile):
    import os
    prefix, _ = os.path.splitext(msfile)
    return prefix

def _list2quotedstr_(list_):
    csv_ = ''
    for itm in list_:
        csv_ += f"'{itm}',"
    return f"[{csv_[:-1]}]"

Fill in flux cal model column
* https://casadocs.readthedocs.io/en/stable/api/tt/casatasks.imaging.setjy.html

In [ ]:
def init_cal(msfile, calibrator, fixvis=False, model='', manual=''):
    if fixvis:
        # phase centre correction
        # Recalculate uvw coordinates so they are consistent with CASA's convention - otherwise images will appear to be inverted
        print(f"fixvis(vis='{msfile}', reuse=False)")
    print(f"clearcal('{msfile}')")
    # apply calibrator model to model column to retain calibrator structure for extended sources
    if model: raise RuntimeError('Add model setjy')
    if manual: raise RuntimeError('Add manual setjy')
    else:
        print(f"setjy(vis='{msfile}', field='{calibrator}', fluxdensity=-1, standard='Perley-Butler 2010')")
    # view calibrator model
    print(f"plotms(vis='{msfile}', xaxis='freq', yaxis='amp', correlation='XX,YY', field='{calibrator}', ydatacolumn='model', avgtime='{dt}', averagedata=True, avgbaseline=True, coloraxis='scan')")


## Delay calibration

* https://casa.nrao.edu/docs/taskref/gaincal-task.html

In [ ]:
def delay_cal(msfile, kcal, ref_ant=''):
    prefix = _get_prefix_(msfile)

    # use delay calibration on primary calibrator to inspect data and identify possible bad antennas or antennas to note
    print('# calculate temporary per scan delay corrections')
    ktable0 = prefix + '.K0'
    print(f"rmtables('{ktable0}')")
    print(f"gaincal(vis='{msfile}', caltable='{ktable0}', field='{kcal}', refant='{ref_ant}', gaintype='K', solint='int', combine='scan,field', minsnr=3.0)")
    # inspect delay solutions and see if they make sense.
    print("# delays after calibration should be no more than a few nanoseconds and spread around/close to zero.")
    print("# per antenna spread should be small as well")
    print(f"plotcal(caltable='{ktable0}', xaxis='antenna', yaxis='delay')")
    print(f"plotcal(caltable='{ktable0}', xaxis='time', yaxis='delay', iteration='antenna', subplot=421)")
    # optional
    # temporarily apply delay cal to check bandpass with delay solution applied
    print(f"applycal(vis='{msfile}', field='{kcal}', gaintable=['{ktable0}'], gainfield=['{kcal}'], interp=['nearest'], calwt=False, applymode='calflag')")
    # check bandpass with delay solutions applied and confirm that there is no fringing.
    print(f"plotms(vis='{msfile}', xaxis='freq', yaxis='phase', correlation='XX,YY', field='{kcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print()

    print('# combine all scans to derive a single delay correction')
    ktable = prefix + '.K'
    print(f"rmtables('{ktable}')")
    # combine all scans to derive a single delay correction
    print(f"gaincal(vis='{msfile}', caltable='{ktable}', field='{kcal}', refant='{ref_ant}', gaintype='K', solint='inf', combine='scan,field', minsnr=3.0)")
    # inspect delay solutions and see if they make sense.    
    print("# delays after calibration should be no more than a few nanoseconds and spread around/close to zero.")
    print(f"plotcal(caltable='{ktable}', xaxis='antenna', yaxis='delay')")
    print("# temporarily apply delay cal to check bandpass with delay solution applied")
    print(f"applycal(vis='{msfile}', field='{kcal}', gaintable=['{ktable}'], gainfield=['{kcal}'], interp=['nearest'], calwt=False, applymode='calflag')")
    print("# check bandpass with delay solutions applied and confirm that there is no fringing.")
    print(f"plotms(vis='{msfile}', xaxis='freq', yaxis='phase', correlation='XX,YY', field='{kcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print()
    
    return ktable

## Bandpass calibration

* https://casa.nrao.edu/docs/taskref/bandpass-task.html

Some decisions need to be made here.    
Because the target field has continuum emission of similar or greater strength to the bandpass calibrator, a noisy per/channel bandpass solution will transfer the noise into the continuum field.  Thus a solution-type of 'BPOLY' should be used or smoothing options investigated.    
The order of the polynomials for amplitude and phase can be set separately.    
This can be determined by inspection of the time-averaged visibility data on the bandpass calibrator, with `gtable0` applied if there is significant phase drift.    
Note that since we will be doing this initial data reduction over ~ 10 MHz, a low order polynomial is expected to be sufficient.

In [ ]:
def bp_cal(msfile, bcal, ref_ant='', ref_chans='', gaintables=[], gainfields=[]):
    prefix = _get_prefix_(msfile)
    gtlist = _list2quotedstr_(gaintables)
    gflist = _list2quotedstr_(gainfields)

    # preliminary gaincal
    print("# a throw-away calibration carried out over a small part of the band where the bandpass response is sufficiently flat")
    gtable0 = prefix + '.G0'
    print(f"rmtables('{gtable0}')")
    # if significant phase drift occurs over the duration of the bandpass calibrator scan, this needs to be corrected prior to averaging over time.
    print(f"gaincal(vis='{msfile}', caltable='{gtable0}', field='{bcal}', refant='{ref_ant}', gaintype='G', spw='{ref_chans}', calmode='p', solint='inf', minsnr=3.0, solnorm=True, gaintable={gtlist}, interp='nearest')")
#     print(f"gaincal(vis='{msfile}', caltable='{gtable0}', field='{bcal}', refant='{ref_ant}', gaintype='G', spw='{ref_chans}', calmode='ap', solint='int', minsnr=3.0, solnorm=True, gaintable={gtlist}, interp='nearest')")
    # inspect temp gain solutions
    print("# antenna-based components which can be inspected for each antenna, all phase solutions should be in a straight line since the flux cal is a point source")
    print(f"plotcal(caltable='{gtable0}', xaxis='time', yaxis='phase', plotrange=[-1, -1, -180, 180], iteration='antennas', subplot=421)")
#     print(f"plotcal(caltable='{gtable0}', xaxis='time', yaxis='amp', iteration='antennas', subplot=421)")
    print()

    # For narrow band and 32k spectral line calibration
    # Note: Because the target field has continuum emission of similar or greater strength to the bandpass calibrator,
    # a noisy per/channel bandpass solution will transfer the noise into the continuum field.
    # Thus a solution-type of 'BPOLY' should be used or smoothing options investigated.
    gtlist0 = f"['{gtable0}',{gtlist[1:-1]}]"
    gflist0 = f"['{bcal}',{gflist[1:-1]}]"
    print("# The order of the polynomials for amplitude and phase can be set separately.")
    print("# This can be determined by inspection of the time-averaged visibility data on the bandpass calibrator, with gtable0 applied if there is significant phase drift.")
    print(f"applycal(vis='{msfile}', field='{bcal}', gaintable={gtlist0}, gainfield={gflist0}, interp=['nearest', 'nearest'], calwt=False, applymode='calflag')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='amp', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print()

    # bandpass calibration
    gflist = f"['{bcal}',{gflist[1:-1]}]"
    # Bandpass does not show time-variation within the observation session, but some antennas have nasty dips and kinks.
    # So combine all scans to improve s/n. Possibly need to do some smoothing
    # Only using the primary calibrator that you have a flux model for (build a model for 0408-65).
    btable = prefix + '.B'
    gtlist = f"['{btable}',{gtlist[1:-1]}]"
    print(f"rmtables('{btable}')")
    print(f"bandpass(vis='{msfile}', caltable='{btable}', field='{bcal}', refant='{ref_ant}', bandtype='B', solint='inf', combine='scan, field', minsnr=3, solnorm=True, gaintable={gtlist0}, interp=['nearest', 'nearest'])")

    print("# passband in amp around 1. and consistent in phase")
    print(f"plotcal(caltable='{btable}', xaxis='chan', yaxis='amp', iteration='antennas', subplot=421)")
    print(f"plotcal(caltable='{btable}', xaxis='chan', yaxis='phase', iteration='antennas', subplot=421)")

    print(f"plotbandpass(caltable='{btable}', yaxis='amp', poln='xx,yy', subplot=11, overlay='time')")
    print(f"plotbandpass(caltable='{btable}', yaxis='phase', poln='xx,yy', subplot=11, overlay='time')")

    print(f"applycal(vis='{msfile}', field='{bcal}', gaintable={gtlist}, gainfield={gflist}, interp=['nearest', 'nearest'], calwt=False, applymode='calflag')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, iteraxis='antenna', coloraxis='corr')")
    print()

    print("# a low order polynomial is expected to be sufficient in NB")
    btable_poly = prefix + '_bpoly.B'
    print(f"rmtables('{btable_poly}')")
    degamp = 50  # 3  # >=3
    degphase = 20  # 5  # >=3
    # maybe do only bpoly on nb data == narrow pb
    print(f"bandpass(vis='{msfile}', caltable='{btable_poly}', field='{bcal}', refant='{ref_ant}', bandtype='BPOLY', solint='inf', combine='scan, field', minsnr=3, solnorm=True, degamp = {degamp}, degphase = {degphase}, gaintable={gtlist0}, interp=['nearest', 'nearest'])")
    gtlist_ = f"['{btable_poly}',{gtlist[1:-1]}]"
    print(f"applycal(vis='{msfile}', field='{bcal}', gaintable={gtlist}, gainfield={gflist}, interp=['nearest', 'nearest'], calwt=False, applymode='calflag')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{bcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, iteraxis='antenna', coloraxis='corr')")
    print()

    return btable

## Complex gain calibration

* https://casa.nrao.edu/docs/taskref/gaincal-task.html

In [ ]:
def phase_cal(msfile, gcal, ref_ant='', append=False, gaintables=[], gainfields=[]):
    prefix = _get_prefix_(msfile)
    gtlist = _list2quotedstr_(gaintables)
    gflist = _list2quotedstr_(gainfields)

    # Now ignore gtable0 and start again with a bandpass correction applied before full gain calibration.
    # use complex gain calibration on primary calibrator to inspect data and identify possible bad time dumps to note
    print('# calculate temporary per dump gain corrections')
    gtable1 = prefix + '.G1'
    print(f"rmtables('{gtable1}')")
    print(f"gaincal(vis='{msfile}', caltable='{gtable1}', field='{gcal}', refant='{ref_ant}', gaintype='G', calmode='ap', solint='int', combine='spw', minsnr=3, solnorm=False, gaintable={gtlist}, interp=['nearest', 'nearest'])")
    print("# phase and amp solutions over a scan must be flat for point sources")
    print(f"plotcal(caltable='{gtable1}', xaxis='time', yaxis='phase', field='{gcal}', plotrange=[-1, -1, -180, 180])")
    print(f"plotcal(caltable='{gtable1}', xaxis='time', yaxis='amp', field='{gcal}')")
    # or
    print(f"plotcal(caltable='{gtable1}', xaxis='time', yaxis='phase', field='{gcal}', plotrange=[-1, -1, -180, 180], iteration='antennas', subplot=421)")
    print(f"plotcal(caltable='{gtable1}', xaxis='time', yaxis='amp', field='{gcal}', iteration='antennas', subplot=421)")
    print()

    print("# We assume that we can set the solution interval to the duration of the scan.")
    gtable = prefix + '.G'
    if not append: print(f"rmtables('{gtable}')")
    print(f"gaincal(vis='{msfile}', caltable='{gtable}', field='{gcal}', refant='{ref_ant}', gaintype='G', calmode='ap', solint='inf', combine='spw', append={append}, minsnr=3, solnorm=False, gaintable={gtlist}, interp=['nearest', 'nearest'])")
    print(f"plotcal(caltable='{gtable}', xaxis='time', yaxis='phase', field='{gcal}', plotrange=[-1, -1, -180, 180])")
    print(f"plotcal(caltable='{gtable}', xaxis='time', yaxis='amp', field='{gcal}')")
    print(f"plotcal(caltable='{gtable}', xaxis='time', yaxis='phase', field='{gcal}', plotrange=[-1, -1, -180, 180], iteration='antennas', subplot=421)")
    print(f"plotcal(caltable='{gtable}', xaxis='time', yaxis='amp', field='{gcal}', iteration='antennas', subplot=421)")

    gtlist = f"['{gtable}',{gtlist[1:-1]}]"
    gflist = f"['{gcal}',{gflist[1:-1]}]"
    print(f"applycal(vis='{msfile}', field='{gcal}', gaintable={gtlist}, gainfield={gflist}, interp=['nearest', 'nearest', 'nearest'], calwt=False, applymode='calflag')")
    print(f"applycal(vis='{msfile}', field='{gcal}', gaintable={gtlist}, gainfield={gflist}, interp=['linear', 'nearest', 'nearest'], calwt=False, applymode='calflag')")
    # Slice in time (look at all channels averaged over time)
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{gcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[0,0,-180,180])")
#    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', field='{gcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, iteraxis='antenna', coloraxis='corr')")
    print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='amp', correlation='XX,YY', field='{gcal}', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    # Slice in frequency (look at all timestamps averaged over channel)
    print(f"plotms(vis='{msfile}', xaxis='time', yaxis='amp', correlation='XX,YY', field='{gcal}', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr')")
    print(f"plotms(vis='{msfile}', xaxis='time', yaxis='phase', correlation='XX,YY', field='{gcal}', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[0,0,-180,180])")
    print()

    return gtable

In [1]:
def FGBGK(msfile, kcal, bcal, gcal, fcal, ref_ant='', ref_chans=''):

    # K
    print('Delay calibration')
    ktable = delay_cal(msfile, kcal, ref_ant=ref_ant)

    # BG
    print('\nBandpass calibration')
    btable = bp_cal(msfile, bcal, ref_ant=ref_ant, ref_chans=ref_chans, gaintables=[ktable], gainfields=[kcal])

    # G3
    print('\nComplex gain calibration, flux cal')
    # gain calibrartion on fluxscale calibrator
    gtable = phase_cal(msfile, kcal, ref_ant=ref_ant, append=False, gaintables=[btable, ktable], gainfields=[bcal, kcal])

    cals = ''
    if bcal != kcal:  cals = bcal
    if gcal != kcal and gcal != bcal:
        if cals: cals += f", {gcal}"
        else: cals = gcal
    if cals:
        print('\nComplex gain, other calibrators')
        gtable = phase_cal(msfile, cals, ref_ant=ref_ant, append=True, gaintables=[btable, ktable], gainfields=[bcal, kcal])

    # Flux
    # convert correlation coefficients to absolute flux densities
    # https://casa.nrao.edu/docs/taskref/fluxscale-task.html
    prefix = _get_prefix_(msfile)
    print('\nFlux calibration')
    ftable = prefix + '.F'
    print(f"rmtables('{ftable}')")
    cals = ''
    if kcal != fcal: cals = kcal
    if bcal != fcal and bcal != kcal:
        if cals: cals += f", {bcal}"
        else: cals = bcal
    if gcal != fcal and gcal != kcal and gcal != bcal:
        if cals: cals += f", {gcal}"
        else: cals = gcal
    if cals:
        print("# scales the gain on the secondary calibrator by the fluxscale")
        print(f"fluxscale(vis='{msfile}', caltable='{gtable}', fluxtable='{ftable}', reference='{fcal}', transfer='{cals}')")
        gtlist = _list2quotedstr_([ftable, btable, ktable])
        gflist = _list2quotedstr_([gcal, bcal, kcal])
        print(f"applycal(vis='{msfile}', field='{cals}', gaintable={gtlist}, gainfield={gflist}, interp=['nearest', 'nearest', 'nearest'], calwt=False, applymode='calflag')")
        print(f"applycal(vis='{msfile}', field='{cals}', gaintable={gtlist}, gainfield={gflist}, interp=['linear', 'nearest', 'nearest'], calwt=False, applymode='calflag')")
        # Slice in time (look at all channels averaged over time)
        print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
        print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='amp', correlation='XX,YY', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
        # Slice in frequency (look at all timestamps averaged over channel)
        print(f"plotms(vis='{msfile}', xaxis='time', yaxis='amp', correlation='XX,YY', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
        print(f"plotms(vis='{msfile}', xaxis='time', yaxis='phase', correlation='XX,YY', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
    else: ftable = ''
    print()

    return ktable, btable, gtable, ftable

# Wideband calibration recipe

In [ ]:
dt = 5200  # s
nchans = 4096
start_freq = 0.890064  # GHz
stop_freq = 1.6681128  # GHz
pb_start_idx = 163
pb_end_idx = 3885

ref_ant = 'm010'
ref_chans = ''  # '0:2000~3000'

In [ ]:
# Concatenate calibrator observation data
print("# calibrator only measurement set")
cal_mslist=f"['J1939-6342.ms', 'J1726-5529.ms']"
msfile = 'g330_calibrators.ms'
print(f"rmtables('{msfile}')")
print(f"concat(vis={cal_mslist}, concatvis='{msfile}', timesort=True)")

primary_cal = 'J1939-6342'  # delay_cal, flux_cal, bp_cal
secondary_cal = 'J1726-5529'  # gain_cal
target = 'G330.89-0.36'

# calibrator only measurement set
rmtables('g330_calibrators.ms')
concat(vis=['J1939-6342.ms', 'J1726-5529.ms'], concatvis='g330_calibrators.ms', timesort=True)


In [ ]:
if ref_chans:
    c_start, c_stop = ref_chans.split(':')[1].split('~')
else:
    c_start=pb_start_idx
    c_stop=pb_end_idx

check_selection(msfile, primary_cal, c_start=c_start, c_stop=c_stop, r_ant=ref_ant)
print()

# calibration recipe
init_cal(msfile, primary_cal)
print()
ktable, btable, gtable, ftable = FGBGK(msfile, kcal=primary_cal, gcal=secondary_cal, bcal=primary_cal, fcal=primary_cal, ref_ant=ref_ant, ref_chans=ref_chans)
print()

# apply calibration
print('# apply calibration solutions')
gtlist = _list2quotedstr_([ftable, btable, ktable])
gflist = _list2quotedstr_([gcal, bcal, kcal])


applycal(vis=msfile,
             field=target,
             gaintable=[ktable0, btable1, ftable1],
             gainfield=[delay_cal,bp_cal, g_cal],
             interp=['nearest', 'nearest', 'linear'])

print('# corrected calibrator data')
# Slice in time (look at all channels averaged over time)
print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='phase', correlation='XX,YY', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
print(f"plotms(vis='{msfile}', xaxis='chan', yaxis='amp', correlation='XX,YY', ydatacolumn='corrected', avgtime='{dt}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
# Slice in frequency (look at all timestamps averaged over channel)
print(f"plotms(vis='{msfile}', xaxis='time', yaxis='amp', correlation='XX,YY', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")
print(f"plotms(vis='{msfile}', xaxis='time', yaxis='phase', correlation='XX,YY', ydatacolumn='corrected', avgchannel='{nchans}', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='field')")



Quick data inspection to confirm ref ant and ref chans before calibration
plotms(vis='g330_calibrators.ms', xaxis='chan', yaxis='amp', correlation='XX,YY', field='J1939-6342', avgtime='5200', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[163, 3885, 0, 0])
plotms(vis='g330_calibrators.ms', xaxis='chan', yaxis='phase', correlation='XX,YY', field='J1939-6342', avgtime='5200', averagedata=True, avgscan=True, avgbaseline=True, coloraxis='corr', plotrange=[163, 3885, -180, 180])
plotms(vis='g330_calibrators.ms', xaxis='time', yaxis='amp', correlation='XX,YY', field='J1939-6342', antenna='m010', avgchannel='4096', averagedata=True, avgscan=True, avgbaseline=False, coloraxis='corr')
plotms(vis='g330_calibrators.ms', xaxis='time', yaxis='phase', correlation='XX,YY', field='J1939-6342', antenna='m010', avgchannel='4096', averagedata=True, avgscan=True, avgbaseline=False, coloraxis='corr')
Switch views interactively as needed, check other sources for obvious issue

NameError: ignored